In [1]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db


In [4]:
year1_minute, meta = Db.load_data(hourly=False, meta=True)

In [3]:
year1_minute

,Timestamp,TimeStamp_Count,DHW_ClothesWasherColdFlow,DHW_ClothesWasherHotFlow,DHW_DishwasherHotFlow,DHW_HeatPumpWaterHeaterEnergyTotal,DHW_HeatPumpWaterHeaterPowerTotal,DHW_ManifoldColdFlow,DHW_ManifoldHotFlow,DHW_MixValveColdFlow,...,Vent_HRVTempReturnIn,Vent_HRVTempSupplyOut,Vent_HRVfreshairpressuredifferential,Vent_HRVstaleairpressuredifferential,HVAC_HeatPumpIndoorUnitPower,HVAC_HeatPumpOutdoorUnitPower,HVAC_DehumidifierAirflow,HVAC_DehumidifierExitAirTemp,HVAC_DehumidifierInletAirTemp,HVAC_DehumidifierPower
0,2013-07-01 00:01:14-04:00,1,0.0,0.0,0.00000,0.000000,6.920000,0.000000,0.00000,0.00000,...,22.552117,23.758415,NaN,NaN,125.4000,922.6568,NaN,NaN,NaN,NaN
1,2013-07-01 00:02:14-04:00,2,0.0,0.0,0.00000,0.132000,6.920000,0.000000,0.00000,0.00000,...,22.523656,23.737489,NaN,NaN,127.0400,919.2424,NaN,NaN,NaN,NaN
2,2013-07-01 00:03:14-04:00,3,0.0,0.0,0.00000,0.264000,6.920000,0.000000,0.00000,0.00000,...,22.539345,23.742217,NaN,NaN,153.3050,920.8184,NaN,NaN,NaN,NaN
3,2013-07-01 00:04:14-04:00,4,0.0,0.0,0.00000,0.396000,6.920000,0.000000,0.00000,0.00000,...,22.539554,23.727526,NaN,NaN,143.4575,922.3944,NaN,NaN,NaN,NaN
4,2013-07-01 00:05:14-04:00,5,0.0,0.0,0.00000,0.528000,6.920000,0.000000,0.00000,0.00000,...,22.506425,23.705258,NaN,NaN,141.8150,925.2832,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,2014-06-30 23:29:22-04:00,518788,0.0,0.0,1.86261,1074.687075,765.920563,11.297412,57.77676,8.69218,...,23.912489,24.423000,2.209914,2.503488,110.6250,1517.2800,NaN,NaN,NaN,NaN
518788,2014-06-30 23:30:22-04:00,518789,0.0,0.0,1.86261,1087.432650,766.929871,11.297412,57.77676,8.69218,...,23.898320,24.403229,2.195995,2.512385,124.5800,1394.6256,NaN,NaN,NaN,NaN
518789,2014-06-30 23:31:22-04:00,518790,0.0,0.0,1.86261,1100.200137,768.525940,11.297412,57.77676,8.69218,...,23.849174,24.392863,2.224423,2.554394,131.1450,1311.6304,NaN,NaN,NaN,NaN
518790,2014-06-30 23:59:22-04:00,518791,0.0,0.0,1.86261,1173.107485,6.920000,11.297412,57.77676,8.69218,...,24.119352,24.579783,2.239691,2.474923,127.0400,1325.5504,NaN,NaN,NaN,NaN


In [5]:
production = meta.loc[(meta['Parameter'] == "Power_Electrical") & (
    meta["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

consumption = meta.loc[((meta['Parameter'] == "Power_Electrical") | (meta['Parameter'] == "Power_Thermal")) & (
    meta["Description"].str.contains("power consumption" or "used"))]["Unnamed: 0"].tolist()

StatusOnOff = meta.loc[meta['Parameter'] == "Status_OnOff"]["Unnamed: 0"].tolist()

In [8]:
year1_minute[StatusOnOff].describe().T

,count,mean,std,min,25%,50%,75%,max
DHW_StatusSolenoidColdKitchenSink,518710.0,0.020073,0.140250,0.00,0.0,0.0,0.0,1.000000
DHW_StatusSolenoidColdMBAShower,518710.0,0.023134,0.167154,-1.11,0.0,0.0,0.0,1.000000
DHW_StatusSolenoidColdMBATub,518710.0,0.000376,0.075611,-1.11,0.0,0.0,0.0,1.000000
DHW_StatusSolenoidHotKitchenSink,518710.0,0.020073,0.140250,0.00,0.0,0.0,0.0,1.000000
DHW_StatusSolenoidHotMBAShower,518710.0,0.023134,0.167154,-1.11,0.0,0.0,0.0,1.000000
DHW_StatusSolenoidHotMBATub,518710.0,-0.000588,0.068942,-1.11,0.0,0.0,0.0,1.000000
Load_StatusApplianceCooktop,518708.0,0.016038,0.145334,-1.11,0.0,0.0,0.0,1.000000
Load_StatusApplianceDishwasher,518708.0,0.065457,0.247331,0.00,0.0,0.0,0.0,1.000000
Load_StatusApplianceOven,518708.0,0.045407,0.208195,0.00,0.0,0.0,0.0,1.000000
Load_StatusApplianceRangeHood,518708.0,0.015995,0.145193,-1.11,0.0,0.0,0.0,1.000000
